# What we do here

- Load items dataframe
- Embed items 
- Compute cosine similarity matrix per scale
- Construct final data frame with the following columns
    - Row =  scale name
    - Col1 = cosine similarity matrix
    - Col2 = scale id


In [11]:
#source Jonas/bin/activate 
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from openai import OpenAI
import torch
import json

# 1 Load items

In [5]:
df = pd.read_csv('items.csv')
df.head()

,item_id,scale_id,item_text,source_table
0,1,360emergencymed_azami_2024,She/he performs the clinical examinations acco...,360emergencymed_azami_2024
1,2,360emergencymed_azami_2024,The student provides accurate and authoritativ...,360emergencymed_azami_2024
2,3,360emergencymed_azami_2024,Uses appropriate communication skills while ta...,360emergencymed_azami_2024
3,4,360emergencymed_azami_2024,"She/he patiently, timely, and correctly perfor...",360emergencymed_azami_2024
4,5,360emergencymed_azami_2024,She/he knows the correct use of the medical eq...,360emergencymed_azami_2024


In [6]:
#drop empty items
df = df[~df['item_text'].isna()]

# 2 Compute Embeddings 

In [12]:
# Example dataframe
# df['item_text']

# List of models (can mix SentenceTransformer and OpenAI models)
models = [
    'nli-distilroberta-base-v2',
    'paraphrase-multilingual-mpnet-base-v2',
    'paraphrase-multilingual-MiniLM-L12-v2',
    'intfloat/multilingual-e5-base',
    'LaBSE',
    
    # --- Add OpenAI embedding models here ---
    'openai:text-embedding-3-large',
    'openai:text-embedding-3-small'
]

# Initialize OpenAI client
client = OpenAI()

# Helper function
def get_openai_embedding(model_name, text):
    """Return embedding using OpenAI API."""
    response = client.embeddings.create(
        model=model_name,
        input=text
    )
    return response.data[0].embedding


# Main loop
for mod in models:
    print(f"Processing model: {mod}")
    item_embed = []

    # Detect if this model is an OpenAI embedding model
    if mod.startswith("openai:"):
        openai_model = mod.replace("openai:", "")
        
        for text in df['item_text']:
            emb = get_openai_embedding(openai_model, text)
            item_embed.append(emb)

    else:
        # SentenceTransformer model
        st_model = SentenceTransformer(mod)

        for text in df['item_text']:
            emb = st_model.encode(text)
            item_embed.append(emb)

    # Save embeddings to dataframe
    df[f"{mod}_embeddings"] = item_embed

Processing model: nli-distilroberta-base-v2
Processing model: paraphrase-multilingual-mpnet-base-v2
Processing model: paraphrase-multilingual-MiniLM-L12-v2
Processing model: intfloat/multilingual-e5-base
Processing model: LaBSE
Processing model: openai:text-embedding-3-large
Processing model: openai:text-embedding-3-small


# 3 construct final dataframe

In [25]:
scales_corr_rows = []
model_short = ['distilroberta', 'mpnet', 'miniLM', 'e5', 'labse', 'gpt3-large', 'gpt3-small']

for scale in df['scale_id'].unique():
    temp_df = df[df['scale_id'] == scale]
    
    row = {"scale_id": scale}
    
    for m_idx in range(len(models)):
        col = models[m_idx] + '_embeddings'
        
        # Convert the Series of vectors to a 2D numpy array
        emb_matrix = np.vstack(temp_df[col].to_list())
        
        # Convert to torch tensor
        emb_tensor = torch.tensor(emb_matrix)
        sim_tensor = util.pytorch_cos_sim(emb_tensor, emb_tensor)

        # Convert to numpy for further handling
        sim_matrix = sim_tensor.numpy()
        
        # Fill diagonal with 1
        np.fill_diagonal(sim_matrix, 1.0)
        
        # ⚠️ IMPORTANT CHANGE: store JSON string, not raw ndarray
        row[model_short[m_idx]] = json.dumps(sim_matrix.tolist())
    
    scales_corr_rows.append(row)

# Final DataFrame: one row per scale, one column per model (plus scale_id)
scales_corr_df = pd.DataFrame(scales_corr_rows)

scales_corr_df.to_csv('cosine_scales.csv', index=False)


In [30]:
sim_matrix.shape

(8, 8)